In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [27]:
train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [28]:
test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip")
test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [29]:
train["Date"] = pd.to_datetime(train["Date"])
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train

,Store,Dept,Date,Weekly_Sales,IsHoliday,Year,Month
0,1,1,2010-02-05,24924.50,False,2010,2
1,1,1,2010-02-12,46039.49,True,2010,2
2,1,1,2010-02-19,41595.55,False,2010,2
3,1,1,2010-02-26,19403.54,False,2010,2
4,1,1,2010-03-05,21827.90,False,2010,3
...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,2012,9
421566,45,98,2012-10-05,628.10,False,2012,10
421567,45,98,2012-10-12,1061.02,False,2012,10
421568,45,98,2012-10-19,760.01,False,2012,10


In [30]:
test["Date"] = pd.to_datetime(test["Date"])
test["Year"] = test["Date"].dt.year
test["Month"] = test["Date"].dt.month
test

,Store,Dept,Date,IsHoliday,Year,Month
0,1,1,2012-11-02,False,2012,11
1,1,1,2012-11-09,False,2012,11
2,1,1,2012-11-16,False,2012,11
3,1,1,2012-11-23,True,2012,11
4,1,1,2012-11-30,False,2012,11
...,...,...,...,...,...,...
115059,45,98,2013-06-28,False,2013,6
115060,45,98,2013-07-05,False,2013,7
115061,45,98,2013-07-12,False,2013,7
115062,45,98,2013-07-19,False,2013,7


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (14,8))
sns.boxplot(train["Month"], train["Weekly_Sales"], showfliers = False)

In [31]:
train2 = train.drop(columns = ["Date", "Weekly_Sales"])
train2

,Store,Dept,IsHoliday,Year,Month
0,1,1,False,2010,2
1,1,1,True,2010,2
2,1,1,False,2010,2
3,1,1,False,2010,2
4,1,1,False,2010,3
...,...,...,...,...,...
421565,45,98,False,2012,9
421566,45,98,False,2012,10
421567,45,98,False,2012,10
421568,45,98,False,2012,10


In [32]:
test2 = test.drop(columns = "Date")
test2

,Store,Dept,IsHoliday,Year,Month
0,1,1,False,2012,11
1,1,1,False,2012,11
2,1,1,False,2012,11
3,1,1,True,2012,11
4,1,1,False,2012,11
...,...,...,...,...,...
115059,45,98,False,2013,6
115060,45,98,False,2013,7
115061,45,98,False,2013,7
115062,45,98,False,2013,7


In [33]:
%%time 
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs = 4) 
rf.fit(train2, train["Weekly_Sales"])
result = rf.predict(test2)
result

CPU times: user 1min 42s, sys: 1.61 s, total: 1min 43s
Wall time: 27.1 s


array([25727.81650048, 25727.81650048, 25727.81650048, ...,
         713.79625869,   713.79625869,   713.79625869])

In [34]:
sub = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip")
sub

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0


In [35]:
sub["Weekly_Sales"] = result
sub

,Id,Weekly_Sales
0,1_1_2012-11-02,25727.816500
1,1_1_2012-11-09,25727.816500
2,1_1_2012-11-16,25727.816500
3,1_1_2012-11-23,21671.795345
4,1_1_2012-11-30,25727.816500
...,...,...
115059,45_98_2013-06-28,749.766940
115060,45_98_2013-07-05,713.796259
115061,45_98_2013-07-12,713.796259
115062,45_98_2013-07-19,713.796259


In [36]:
sub.to_csv("sub.csv", index = False) # 기본값은 index = 1,true 로 설정되어있는데 이렇게되면 우리 데이터에 있는 첫번째 인덱스 칼럼이 새로운 칼럼이 돼서 우리가 제출해야할 칼럼은 2개인데 칼럼 3개가 되어버린다. 